In [1]:
%cd /Users/natalipeeva/Documents/GitHub/Automatic-Answering-of-City-Council-Questions/

/Users/natalipeeva/Documents/GitHub/Automatic-Answering-of-City-Council-Questions


In [2]:
import sys
import os
src_dir = os.path.join(os.getcwd(), 'src')
sys.path.append(src_dir)

from read_data.read_data import read_urls_questions, get_questions, get_url_content_tuples, get_relevant_docs
from elasticsearch import Elasticsearch
from retrieval.sparse_retrieval.bm25 import set_index, get_result_tuples

In [3]:
es_client = Elasticsearch("http://localhost:9200")

In [36]:
collected, questions =  read_urls_questions(os.path.join('data/reference_urls/reference_urls_collected.csv'),
                                            os.path.join('data/question_answer/questions_updated_urls.csv'), clean_url_nan=True) # read collected urls and questions + remove unsuccessful collection

In [5]:
df_filtered = questions[~questions['URLs'].apply(lambda urls: any(url.startswith("https://www.rijksoverheid.nl/") for url in urls))]

In [22]:
import ast

In [23]:
urls_lists = []
for i in range(len(questions)):
        urls = list(ast.literal_eval(questions['URLs'][i]))
        urls_lists.append(urls)

In [24]:
questions['Cleaned_URLs'] = urls_lists

In [37]:
# Assuming your DataFrame is named 'questions' and the column with URLs is named 'URLs'
filtered_questions = questions[questions['Cleaned_URLs'].apply(lambda urls: any(url.startswith('https://www.amsterdam.nl') for url in urls))]

# The 'filtered_questions' DataFrame will contain only the questions with at least one URL starting with 'https://amsterdam.nl'
col_urls = set(list(collected['URL'])) # check how many common items i have - unfortunately only 20
list_of_lists= list(filtered_questions['Cleaned_URLs'])
flattened_list = [item for sublist in list_of_lists for item in sublist]
# Assuming your two lists are named 'list1' and 'list2'
common_items = set(col_urls).intersection(flattened_list)

filtered_questions2 = questions[questions['URLs'].apply(lambda urls: any(url in urls for url in common_items))]

In [48]:
filtered = collected[collected['URL'].str.startswith('https://www.amsterdam.nl')]

In [49]:
filtered

,Unnamed: 0,URL,Content,Textual_Content,Exception
1,1,https://www.amsterdam.nl/veelgevraagd/?caseid=...,"<!DOCTYPE html>\n\n<html lang=""nl"">\n<head pre...",\n\n\n\nMag ik in Amsterdam zwemmen in open wa...,NaN
8,8,https://www.amsterdam.nl/zorg-ondersteuning/on...,"<!DOCTYPE html>\n\n<html lang=""nl"">\n<head pre...",\n\n\n\nOngedocumenteerden - Gemeente Amsterda...,NaN
9,9,https://www.amsterdam.nl/zorg-ondersteuning/on...,"<!DOCTYPE html>\n\n<html lang=""nl"">\n<head pre...",\n\n\n\nOngedocumenteerden - Gemeente Amsterda...,NaN
15,15,https://www.amsterdam.nl/wonen-leefomgeving/gr...,"<!DOCTYPE html>\n\n<html lang=""nl"">\n<head pre...",\n\n\n\nGroene stad - Gemeente Amsterdam\n\n\n...,NaN
17,17,https://www.amsterdam.nl/privacy/loket/,"<!DOCTYPE html>\n\n<html lang=""nl"">\n<head pre...",\n\n\n\nLoket persoonsgegevens gemeente Amster...,NaN
...,...,...,...,...,...
199,199,https://www.amsterdam.nl/veelgevraagd/?product...,"<!DOCTYPE html>\n\n<html lang=""nl"">\n<head pre...",\n\n\n\nFilmen en fotograferen in de gemeente ...,NaN
202,202,https://www.amsterdam.nl/sociaaldomein/zorgpro...,"<!DOCTYPE html>\n\n<html lang=""nl"">\n<head pre...",\n\n\n\nAdviseurs sociale basis (preventie HGK...,NaN
203,203,https://www.amsterdam.nl/nrga/,"<!DOCTYPE html>\n\n<html lang=""nl"">\n<head pre...",\n\n\n\nNieuwe Rechtspositieregeling Gemeente ...,NaN
204,204,https://www.amsterdam.nl/nieuws/kennisgevingen...,"<!DOCTYPE html>\n\n<html lang=""nl"">\n<head pre...",\n\n\n\nKennisgevingen en bekendmakingen - Gem...,NaN


In [53]:
filtered_questions.to_csv('data/question_answer/amsterdam_questions.csv')

In [38]:
list(filtered_questions['Cleaned_URLs'])

[['https://www.amsterdam.nl/veelgevraagd/?caseid=%7BD6E280FB-4A76-40A0-9B88-12B87E446FA6%7D',
  'https://www.ggd.amsterdam.nl/gezond-wonen/zwemmen-open-water/'],
 ['https://www.amsterdam.nl/zorg-ondersteuning/ondersteuning/vluchtelingen/ongedocumenteerden/'],
 ['https://www.amsterdam.nl/zorg-ondersteuning/ondersteuning/vluchtelingen/ongedocumenteerden/'],
 ['https://www.amsterdam.nl/wonen-leefomgeving/groene-stad'],
 ['https://www.amsterdam.nl/privacy/loket/'],
 ['https://www.amsterdam.nl/veelgevraagd/?productid='],
 ['https://www.amsterdam.nl/projecten/noorderpark/'],
 ['https://www.amsterdam.nl/nieuwsbrieven/bestuur-organisatie/gemeenteraad/dagmails/2020/juni/5-juni-2020/?utm_source=nieuwsbrief&utm_medium=e-mail&utm_term=20200605&utm_content=link_ID0AS5BI055BI&utm_campaign=Dagmail%2520gemeenteraad%2520Amsterdam%252'],
 ['https://www.amsterdam.nl/sociaaldomein/zorg-jeugd/artikelen/specialistische-jeugdhulp-speciaal/'],
 ['https://www.amsterdam.nl/zorg-ondersteuning/ondersteuning/dak-o

In [8]:
question_list = get_questions(filtered_questions2)
document_list = get_url_content_tuples(collected)
mappings = {
        "properties": {
            "url": {"type": "text"},
            "text": {"type": "text", "analyzer": "standard", "similarity": "BM25"}
    }
}

In [9]:
es_client = set_index(es_client=es_client, collected=collected, mappings=mappings) # 48 secs

In [60]:
results = get_result_tuples(es_client=es_client, questions=question_list, n=10) # 15 gives the best results ; when i set it to 5 gives 0,26

In [61]:
from irmetrics.topk import recall

In [62]:
predictions = []
for question in results.keys():
    urls = []
    for result in results[question][1]:
        urls.append(result['url'])
    predictions.append(list(set(urls)))

In [63]:
relevant_docs = get_relevant_docs(df_filtered)

In [64]:
true = []
for question in relevant_docs.keys():
    true.append(list(set(relevant_docs[question])))

In [65]:
import numpy as np
from irmetrics.topk import rr

# Calculate the Mean Reciprocal Rank for each question
mrr_values = []
for i in range(len(predictions)):
    true_values = true[i]
    mrr = rr(true_values, predictions[i], k =10)
    mrr_values.append(mrr)

# Calculate the average Mean Reciprocal Rank
average_mrr = np.mean(mrr_values)

print("Average Mean Reciprocal Rank:", average_mrr)


Average Mean Reciprocal Rank: 0.017080745341614908


In [66]:
from irmetrics.topk import recall

# Calculate the Mean Reciprocal Rank for each question
mrr_values = []
for i in range(len(predictions)):
    true_values = true[i]
    mrr = recall(true_values, predictions[i], k=100)
    mrr_values.append(mrr)

# Calculate the average Mean Reciprocal Rank
average_recall = np.mean(mrr_values)

print("Average Mean Reciprocal Rank:", average_recall)

Average Mean Reciprocal Rank: 0.13043478260869565
